In [1]:
import numpy as np
from tensorflow import keras

In [2]:
from keras import Input, layers
from keras.models import Sequential, Model

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

Using TensorFlow backend.


In [3]:
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)
model = Model(input_tensor, output_tensor)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


## 7.1.2 Multi-input models

* 関数型の API を使えば，複数の入力があるニューラルネットワークを構築することができる．
* `keras.layers.add` や `keras.layers.concatenate` などで結合する

In [14]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

# text を 64 次元のベクトルに埋め込むレイヤ
embedded_text = layers.Embedding(
    input_dim=text_vocabulary_size,
    output_dim=64)(text_input)

# 埋め込まれたベクトルのコレクション (BachSize, 64) を，ひとつの 32 次元ベクトルに変換
encoded_text = layers.LSTM(32)(embedded_text)

# question についても同様に
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# encode された２つのベクトルを結合する
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# 全結合層で最終出力
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# テンソル answer の計算グラフでモデルを構築
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
             metrics=['acc'])

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LS

In [65]:
# データを作る

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

In [67]:
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.5669 - acc: 0.0270
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.5024 - acc: 0.0370
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.4214 - acc: 0.0480
Epoch 4/10
1000/1000 [==============================] - 4s 4ms/step - loss: 5.3517 - acc: 0.0510
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.2970 - acc: 0.0590
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.2282 - acc: 0.0680
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.1693 - acc: 0.0780
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.1025 - acc: 0.0940
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.0410 - acc: 0.1090
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 4.9827 - acc: 0.1320


## 7.1.3 Multi-output models

In [88]:
vocabulary_size = 50000
n_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPool1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# この x : shape=(?, 128) から各属性を予測する
age_predection = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(n_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

# 単純に outputs=[...] とすれば良い
model = Model(posts_input, [age_predection, income_prediction, gender_prediction])

In [98]:
# ロス関数の指定はディクショナリかリストで渡す

loss_list = ['mse', 'categorical_crossentropy', 'binary_crossentropy']
loss_list_dict = {name:value for name,value in zip(model.output_names, loss_list)}

loss_weights_list = [0.25, 1, 10]
loss_weights_dict = {name:value for name,value in zip(model.output_names, loss_weights_list)}

model.compile(optimizer='rmsprop', loss=loss_list, loss_weights=loss_weights_list)

## Inception

GoogLeNet で使用された *inception* モデルの構築方法

#### 1x1 の CNN

* 入力層が $W\times H\times K$ なら，出力層は $W\times H\times 1$
* パラメタ数は $1\times 1\times K$
* チャネル方向での結合を行うので，チャネルの削減に使われる（チャネル削減した後に Conv したり）

In [144]:
x = Input(shape=(100,100,3))
x = layers.Conv2D(128, 5, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(pool_size=2, strides=2)(x)

x

<tf.Tensor 'max_pooling2d_9/MaxPool:0' shape=(?, 50, 50, 128) dtype=float32>

In [153]:
y = Input(shape=(100,100,3))
y = layers.Conv2D(128, 5, activation='relu', padding='same')(y)
y = layers.Conv2D(100, 1, strides=2, padding='same')(y)

y

<tf.Tensor 'conv2d_31/BiasAdd:0' shape=(?, 50, 50, 100) dtype=float32>

In [156]:
layers.add([x, y], axis=-1)

TypeError: ('Keyword argument not understood:', 'axis')

In [126]:
Input?